In [1]:
# 1. 라이브러리 설치
!pip install biopython transformers

# 2. 저장할 디렉토리 생성
import os
os.makedirs('./data/genome', exist_ok=True)

# 3. 인간 유전체 데이터(GRCh38) 다운로드 (약 900MB, 압축 풀면 3GB+)
!wget -O ./data/genome/GRCh38.fna.gz \
 ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.28_GRCh38.p13/GCA_000001405.28_GRCh38.p13_genomic.fna.gz

print("✅ Download Complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.2 MB/s eta 0:00:00
--2025-12-01 13:55:23--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.28_GRCh38.p13/GCA_000001405.28_GRCh38.p13_genomic.fna.gz
           => ‘./data/genome/GRCh38.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCA/000/001/405/GCA_000001405.28_GRCh38.p13 ... done.
==> SIZE GCA_000001405.28_GRCh38.p13_genomic.fna.gz ... 964899914
==> PASV ... done.    ==> RETR GCA_000001405.28_GRCh38.p13_genomic.fna.gz ... done.
Length: 964899914 (920M) (unauthoritative)

GCA_000001405.28_GR 100%[===================>] 920.20M  24.2MB/s    in 90s     

2025-12-01 13:56:54 (10.3 MB/s) - ‘./data/genome/GRCh38.fna.gz’ saved [964899

In [4]:
import gzip
import csv
from Bio import SeqIO
from tqdm.auto import tqdm

# --- 설정 파라미터 ---
INPUT_FILE = './data/genome/GRCh38.fna.gz'     # 다운로드된 파일
OUTPUT_CSV = './data/human_genome_train.csv'   # 저장될 파일

MAX_SEQ_LEN = 1024      # 길이 1024
STRIDE = 1024           # 겹치지 않게 자르기
MAX_SAMPLES = 20000    # 원하는 샘플 수
MAX_N_RATIO = 0.1       # N 비율 10% 커트라인

def create_clean_csv():
    print(f"🚀 Generating Clean CSV: {OUTPUT_CSV}")
    print("Format: [ID, seq]") # 사용자가 원하는 2컬럼 포맷 확인

    with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)

        # 1. 헤더 작성 (요청하신 대로 'ID'와 'seq'만 남김)
        writer.writerow(["ID", "seq"])

        count = 0
        skipped_n = 0

        with gzip.open(INPUT_FILE, "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                chromosome = record.id
                full_seq = str(record.seq).upper()
                seq_len = len(full_seq)

                # 조각내기
                for i in range(0, seq_len, STRIDE):
                    chunk = full_seq[i : i + MAX_SEQ_LEN]

                    if len(chunk) < MAX_SEQ_LEN:
                        continue

                    # N 비율 필터링
                    if (chunk.count('N') / len(chunk)) > MAX_N_RATIO:
                        skipped_n += 1
                        continue

                    # 2. ID 생성 (chrom 정보는 ID에 합쳐버림)
                    # 예: 'CM000663.2'와 위치 '0' -> 'CM000663.2_0'
                    unique_id = f"{chromosome}_{i}"

                    # 3. [ID, seq] 형태로만 저장 (중간 컬럼 없음!)
                    writer.writerow([unique_id, chunk])
                    count += 1

                    if count % 5000 == 0:
                        print(f"Saved {count} rows...", end='\r')

                    if count >= MAX_SAMPLES:
                        print(f"\n✅ Target reached: {MAX_SAMPLES} rows.")
                        return

    print(f"\n✨ Done! File saved at: {OUTPUT_CSV}")
    print(f"Total rows: {count}")
    print(f"Skipped garbage: {skipped_n}")

# 실행
create_clean_csv()

🚀 Generating Clean CSV: ./data/human_genome_train.csv
Format: [ID, seq]
Saved 20000 rows...
✅ Target reached: 20000 rows.


In [5]:
import pandas as pd

# 저장된 파일의 상위 5개만 읽어서 확인
df_check = pd.read_csv('./data/human_genome_train.csv', nrows=5)
print("📄 Data Preview:")
print(df_check.head())



📄 Data Preview:
                 ID                                                seq
0  CM000663.2_10240  TAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTA...
1  CM000663.2_11264  TGCTTAGAGTGGTGGCCAGCGCCCCCTGCTGGCGCCGGGGCACTGC...
2  CM000663.2_12288  CTGGACCCCTACCTGCCGTCTGCTGCCATCGGAGCCCAAAGCCGGG...
3  CM000663.2_13312  TGGATCTGAGCCCTGGTGGAGGTCAAAGCCACCTTTGGTTCTGCCA...
4  CM000663.2_14336  AGTCACTGACCCCGAGACGTTTGCATCCTGCACAGCTAGAGATCCT...
